In [1]:
import lightgbm as lgb
import neptune
import neptune.integrations.optuna as optuna_utils
import optuna
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [2]:
data = fetch_california_housing(return_X_y=True)

In [3]:
def objective(trial):
    data, target = fetch_california_housing(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mse',
        'verbosity': -1,
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', -1, 50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(test_x)
    return mse(test_y, preds) 


# Create a Neptune run
run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/optuna",
    tags=["quickstart", "study", "script"],
)

# Create a NeptuneCallback for Optuna
neptune_callback = optuna_utils.NeptuneCallback(run)

# Pass NeptuneCallback to Optuna Study .optimize()
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, callbacks=[neptune_callback])

# Stop logging to a Neptune run
run.stop()

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/common/optuna/e/NEP1-29142


C:\Users\Sarang\anaconda3\Lib\site-packages\neptune\internal\utils\git.py:71: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")
[I 2024-08-14 14:32:46,395] A new study created in memory with name: no-name-bc798a40-ea10-4a5e-ae0a-465c7f512d73
[I 2024-08-14 14:32:46,613] Trial 0 finished with value: 1.3537379658315822 and parameters: {'learning_rate': 3.2070810286116916e-06, 'num_leaves': 44, 'max_depth': 39, 'min_data_in_leaf': 54, 'feature_fraction': 0.9845737507571285, 'bagging_fraction': 0.7824681879880189, 'bagging_freq': 2, 'lambda_l1': 9.034791243567664, 'lambda_l2': 0.24911457596254794}. Best is trial 0 with value: 1.3537379658315822.
[W 2024-08-14 14:32:47,475] Param bagging_freq unique value length is less than 2.
[W 2024-08-14 14:32:47,475] Param feature_fraction unique value length is less than 2.
[W 2024-08-14 14:32:47,477] Param lambda_l1 unique value length is less than 2.
[W 2024-08-14 14:32:47,477] Param lambda_l2 uniqu

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 424 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] Still waiting for the remaining 168 operations (60.38% done). Please wait.
[neptune] [info   ] All 424 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/common/optuna/e/NEP1-29142/metadata
